<a href="https://colab.research.google.com/github/ancestor9/Data-Analyst-with-Gemini-/blob/main/9%EC%9D%BC%EC%B0%A8/AdventureSales_ETL_datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Tutorial: From Excel workbook to a Power BI report in Microsoft Teams](https://learn.microsoft.com/en-us/power-bi/create-reports/service-from-excel-to-stunning-report)**

<img src='https://learn.microsoft.com/en-us/power-bi/create-reports/media/service-from-excel-to-stunning-report/power-bi-financial-report-service.png'>

## **1. 데이터 다운로드 하기**

>> #### 1.1. [직접 다운받기](https://github.com/microsoft/powerbi-desktop-samples/blob/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx)


>> #### 1.2. Linux Bash 명령어로 다운 받는 법

In [2]:
# Linux Bash 명령어
! wget https://github.com/microsoft/powerbi-desktop-samples/raw/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx

--2025-06-22 12:52:58--  https://github.com/microsoft/powerbi-desktop-samples/raw/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/microsoft/powerbi-desktop-samples/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx [following]
--2025-06-22 12:52:58--  https://raw.githubusercontent.com/microsoft/powerbi-desktop-samples/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14322931 (14M) [application/octet-stream]
Saving to: ‘AdventureWorks Sales.xlsx’

Advent

In [3]:
# prompt: AdventureWorks Sales.xlsx는 7개의 시트(고객, 상품, 지역, 판매 등)를 모두 읽고 사전형태로 데이터를 저장해 다오
# To read multiple sheets from a single Excel file, we will use the pandas library.
# Below is an example of how you can read all sheets from an Excel file into a dictionary of DataFrames.

import pandas as pd

# Function to read all sheets from an Excel file
def read_excel_sheets(excel_file):
    # Using sheet_name=None reads all sheets, each sheet as a DataFrame in a dictionary
    sheets_dict = pd.read_excel(excel_file, sheet_name=None)
    return sheets_dict

sheets_dict = read_excel_sheets('/content/AdventureWorks Sales.xlsx')
sheets_dict.keys()

dict_keys(['Sales Order_data', 'Sales Territory_data', 'Sales_data', 'Reseller_data', 'Date_data', 'Product_data', 'Customer_data'])

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*9UNgxNIu8-HgaxLa.png'>


>> #### 1.3. Python으로 다운 받는 방법

In [4]:
# prompt: https://github.com/microsoft/powerbi-desktop-samples/blob/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx 파일은 여러 시트로 구성되어 있는데 모든 시트를 읽어줘

import pandas as pd

#!pip install pandas openpyxl


# Replace with the actual URL of your Excel file
url = "https://github.com/microsoft/powerbi-desktop-samples/raw/main/AdventureWorks%20Sales%20Sample/AdventureWorks%20Sales.xlsx"

try:
  xls = pd.ExcelFile(url)
  sheet_names = xls.sheet_names

  all_sheets_data = {}
  for sheet_name in sheet_names:
    df = pd.read_excel(url, sheet_name=sheet_name)
    print(df.head())
    all_sheets_data[sheet_name] = df
    print(f"Sheet '{sheet_name}' loaded successfully.")

except Exception as e:
  print(f"Error loading Excel file: {e}")

    Channel  SalesOrderLineKey Sales Order Sales Order Line
0  Reseller           43659001     SO43659      SO43659 - 1
1  Reseller           43659002     SO43659      SO43659 - 2
2  Reseller           43659003     SO43659      SO43659 - 3
3  Reseller           43659004     SO43659      SO43659 - 4
4  Reseller           43659005     SO43659      SO43659 - 5
Sheet 'Sales Order_data' loaded successfully.
   SalesTerritoryKey     Region        Country          Group
0                  1  Northwest  United States  North America
1                  2  Northeast  United States  North America
2                  3    Central  United States  North America
3                  4  Southwest  United States  North America
4                  5  Southeast  United States  North America
Sheet 'Sales Territory_data' loaded successfully.
   SalesOrderLineKey  ResellerKey  CustomerKey  ProductKey  OrderDateKey  \
0           43659001          676           -1         349      20170702   
1           43659002

In [5]:
all_sheets_data

{'Sales Order_data':          Channel  SalesOrderLineKey Sales Order Sales Order Line
 0       Reseller           43659001     SO43659      SO43659 - 1
 1       Reseller           43659002     SO43659      SO43659 - 2
 2       Reseller           43659003     SO43659      SO43659 - 3
 3       Reseller           43659004     SO43659      SO43659 - 4
 4       Reseller           43659005     SO43659      SO43659 - 5
 ...          ...                ...         ...              ...
 121248  Internet           75122001     SO75122      SO75122 - 1
 121249  Internet           75122002     SO75122      SO75122 - 2
 121250  Internet           75123001     SO75123      SO75123 - 1
 121251  Internet           75123002     SO75123      SO75123 - 2
 121252  Internet           75123003     SO75123      SO75123 - 3
 
 [121253 rows x 4 columns],
 'Sales Territory_data':     SalesTerritoryKey          Region         Country          Group
 0                   1       Northwest   United States  North Am

## **2. 데이터베이스에 저장하기**

In [6]:
# prompt: all_sheets_data의 모든 데이터프레임을 sqlite3를 사용하여 DB를 만들고 각 테이블로 저장하라

import sqlite3

# Create a database connection
conn = sqlite3.connect('adventure_sales.db')

# Iterate through the dictionary of DataFrames and save each to a table
for sheet_name, df in all_sheets_data.items():
  # Replace spaces in sheet names with underscores for valid table names
  table_name = sheet_name.replace(" ", "_")
  df.to_sql(table_name, conn, index=False, if_exists='replace')
  print(f"DataFrame for sheet '{sheet_name}' saved to table '{table_name}'.")

# Close the connection
conn.close()

DataFrame for sheet 'Sales Order_data' saved to table 'Sales_Order_data'.
DataFrame for sheet 'Sales Territory_data' saved to table 'Sales_Territory_data'.
DataFrame for sheet 'Sales_data' saved to table 'Sales_data'.
DataFrame for sheet 'Reseller_data' saved to table 'Reseller_data'.
DataFrame for sheet 'Date_data' saved to table 'Date_data'.
DataFrame for sheet 'Product_data' saved to table 'Product_data'.
DataFrame for sheet 'Customer_data' saved to table 'Customer_data'.


In [7]:
# prompt: with 구문으로 코드를 만들어주라

# Create a database connection using the with statement
with sqlite3.connect('adventure_sales.db') as conn:
    # Iterate through the dictionary of DataFrames and save each to a table
    for sheet_name, df in all_sheets_data.items():
      # Replace spaces in sheet names with underscores for valid table names
      table_name = sheet_name.replace(" ", "_")
      df.to_sql(table_name, conn, index=False, if_exists='replace')
      print(f"DataFrame for sheet '{sheet_name}' saved to table '{table_name}'.")


DataFrame for sheet 'Sales Order_data' saved to table 'Sales_Order_data'.
DataFrame for sheet 'Sales Territory_data' saved to table 'Sales_Territory_data'.
DataFrame for sheet 'Sales_data' saved to table 'Sales_data'.
DataFrame for sheet 'Reseller_data' saved to table 'Reseller_data'.
DataFrame for sheet 'Date_data' saved to table 'Date_data'.
DataFrame for sheet 'Product_data' saved to table 'Product_data'.
DataFrame for sheet 'Customer_data' saved to table 'Customer_data'.


In [11]:
# prompt: Sales_data의 내용을 retrive해줘

import pandas as pd
# Create a database connection
conn = sqlite3.connect('adventure_sales.db')

# Read the data from the Sales_Data table
sales_data_df = pd.read_sql_query("SELECT * FROM Sales_Data", conn)

# Close the connection
conn.close()

# Display the retrieved data
sales_data_df.head()


,SalesOrderLineKey,ResellerKey,CustomerKey,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,SalesTerritoryKey,Order Quantity,Unit Price,Extended Amount,Unit Price Discount Pct,Product Standard Cost,Total Product Cost,Sales Amount
0,43659001,676,-1,349,20170702,20170712,20170709.0,5,1,2024.994,2024.994,0,1898.0944,1898.0944,2024.994
1,43659002,676,-1,350,20170702,20170712,20170709.0,5,3,2024.994,6074.982,0,1898.0944,5694.2832,6074.982
2,43659003,676,-1,351,20170702,20170712,20170709.0,5,1,2024.994,2024.994,0,1898.0944,1898.0944,2024.994
3,43659004,676,-1,344,20170702,20170712,20170709.0,5,1,2039.994,2039.994,0,1912.1544,1912.1544,2039.994
4,43659005,676,-1,345,20170702,20170712,20170709.0,5,1,2039.994,2039.994,0,1912.1544,1912.1544,2039.994


## **EDA, 시각화, RFM 분석**